In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [5]:
INPUT_BASE_PATH = "/kaggle/input/m5-forecasting-accuracy"
OUTPUT_BASE_BATH = "/kaggle/working"

CALENDAR_DATA = pd.read_csv(f"{INPUT_BASE_PATH}/calendar.csv")
# SELL_PRICES = pd.read_csv(f"{INPUT_BASE_PATH}/sell_prices.csv")
# SALES_TRAIN_EVALUATION = pd.read_csv(f"{INPUT_BASE_PATH}/sales_train_evaluation.csv")
SALES_TRAIN_VALIDATION = pd.read_csv(f"{INPUT_BASE_PATH}/sales_train_validation.csv")
SAMPLE_SUBMISSION = pd.read_csv(f"{INPUT_BASE_PATH}/sample_submission.csv")

# PREPROCESSING & FEATURE ENGINEERING

In [3]:
def flatten_sales_df(sales_df: pd.DataFrame, timestamp_cols: list[str]) -> pd.DataFrame:
    sales_dfs = []
    grouper = train_df.groupby(
        [
            "item_id",
            "dept_id",
            "cat_id",
            "store_id",
            "state_id"
        ]
    )
    for key, sales_df in tqdm(grouper):
        (item, dept, cat, store, state) = key
        sales_df = sales_df[timestamp_cols].T.reset_index()
        sales_df.columns = ["d", "count"]
        sales_df = sales_df.assign(
            d = sales_df["d"].apply(lambda d: int(d.lstrip("d_"))),
#             item_id = item,
            dept_id = dept,
            cat_id = cat,
            store_id = store,
            state_id = state,
        )
        sales_dfs.append(sales_df)
    return pd.concat(sales_dfs)


def format_calendar_features(calendar_data: pd.DataFrame) -> pd.DataFrame:
    ...
        

def merge_sales_and_calendar_data(sales_df: pd.DataFrame, calendar_data: pd.DataFrame) -> pd.DataFrame:
    return sales_df.merge(right=calendar_data, on="d", how="left")

In [ ]:
timestamp_cols = [f"d_{i}" for i in range(1, 1914)]

train_df = SALES_TRAIN_VALIDATION.copy(deep=True)
train_df = flatten_sales_df(train_df, timestamp_cols)
# train_df = merge_sales_and_calendar_data(train_df, CALENDAR_DATA)

In [ ]:
MAX_VALIDATION_TIMESTAMP = 1913
VALIDATION_PERIOD = 28
N_FOLDS = 10

for fold_idx in range(N_FOLDS):
    start = MAX_VALIDATION_TIMESTAMP - (N_FOLDS - fold_idx) * VALIDATION_PERIOD
    stop = start + VALIDATION_PERIOD
    
    # Select train data and valid data
    train_data = train_df[train_df["d_int"] <= start].copy()
    valid_data = train_df[(train_df["d_int"] > start) & (train_df["d_int"] <= stop)].copy()

    break

In [12]:
"d_1".lstrip("d_")

'1'